In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132502")
exp = Experiment(workspace=ws, name="quick-starts-experiment")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FLRL4SS7F to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-132502
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-132502


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# the code sninppet is based on official Microsoft documentation, which can be found
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python
from azureml.core.compute_target import ComputeTargetException

# A name for the cluster
cpu_cluster_name = "Udacity-Lab1"

# Verification that cluster does not exist yet
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('The cluster already exists')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": choice(0.001, 0.01, 0.1, 1.0),
        "--max_iter": choice(30, 50, 100, 250)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=50, slack_factor=None, slack_amount=0.2, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./", compute_target=cpu_cluster, entry_script="train.py", script_params=None)
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, 
                                    max_concurrent_runs=4, policy=policy, estimator=est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_submit = exp.submit(hyperdrive_config)

In [5]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_submit).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [16]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# the code is based on the official Microsoft documentation, which can be found
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#find-the-best-model 
best_run = hyperdrive_submit.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_f47c8cd9-b86d-4829-b559-e22875b641f1_10

 Accuracy: 0.9130954570535221


In [17]:
best_model = best_run.register_model(model_name="best_model", model_path="./")

In [19]:
print(best_run)

Run(Experiment: quick-starts-experiment,
Id: HD_f47c8cd9-b86d-4829-b559-e22875b641f1_10,
Type: azureml.scriptrun,
Status: Completed)


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
datastore_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)
ds.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
train_dataset = pd.concat([x_train, y_train], axis=1)

In [10]:
train_dataset

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
21492,35,0,0,1,0,5,4,49,1,999,...,0,0,0,0,0,0,1,0,0,0
13344,34,1,0,1,0,6,1,78,2,999,...,0,0,0,0,1,0,0,0,0,0
11571,52,1,0,1,0,6,4,60,3,999,...,1,0,0,0,0,0,0,0,1,0
2135,35,1,0,0,1,5,4,220,1,999,...,1,0,0,1,0,0,0,0,0,0
733,32,1,0,0,1,8,4,81,1,999,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29460,32,0,0,0,0,8,3,130,3,999,...,0,0,0,0,0,0,0,1,0,0
29699,37,1,0,1,0,5,1,351,1,999,...,1,0,0,0,0,0,0,1,0,0
23422,56,1,0,1,0,7,3,90,1,999,...,1,0,1,0,0,0,0,0,0,0
17906,39,1,0,0,0,5,5,143,2,999,...,0,0,0,1,0,0,0,0,0,0


In [11]:
from azureml.core import Dataset, Datastore

# the code is based on the official Microsoft documentation, which can be found 
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#create-a-dataset-from-pandas-dataframe 

local_path = './prepared.csv'
train_dataset.to_csv(local_path)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='./', target_path='./')

# create a dataset referencing the cloud location
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('./prepared.csv'))])

Uploading an estimated of 5 files
Uploading ./prepared.csv
Uploaded ./prepared.csv, 1 files out of an estimated total of 5
Uploading ./train.py
Uploaded ./train.py, 2 files out of an estimated total of 5
Uploading ./udacity-project.ipynb
Uploaded ./udacity-project.ipynb, 3 files out of an estimated total of 5
Uploading ./.ipynb_aml_checkpoints/udacity-project-checkpoint2020-11-30-15-22-0.ipynb
Uploaded ./.ipynb_aml_checkpoints/udacity-project-checkpoint2020-11-30-15-22-0.ipynb, 4 files out of an estimated total of 5
Uploading ./__pycache__/train.cpython-36.pyc
Uploaded ./__pycache__/train.cpython-36.pyc, 5 files out of an estimated total of 5
Uploaded 5 files


In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=cpu_cluster)

In [13]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
automl_submit = exp.submit(automl_config)
RunDetails(automl_submit).show()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [27]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
import joblib

best_automl_run, best_automl_model = automl_submit.get_output()

print('Best Run Id: ', best_automl_run.id)

best_automl_run.register_model(model_name="best_automl_model", model_path="./")

Best Run Id:  AutoML_a2daf02e-ac90-410a-9224-85c9cd77d114_23


Model(workspace=Workspace.create(name='quick-starts-ws-132502', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-132502'), name=best_automl_model, id=best_automl_model:2, version=2, tags={}, properties={})

In [28]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_automl_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['1', '0', '20', '10', '17', '4'],
 'weights': [0.13333333333333333,
             0.4666666666666667,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.13333333333333333]}

1 - maxabsscaler
{'copy': True}

1 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_stat

In [29]:
print(best_automl_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_impurity_split=None,
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.01,
                    

In [30]:
cpu_cluster.delete()